# Imports

In [ ]:
# google operations
from google.colab import userdata
from google.cloud.bigquery import magics

# data
import numpy as np
import pandas as pd
import datetime as dt

# ML
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support, roc_auc_score

In [ ]:
%load_ext bigquery_magics

In [ ]:
PROJECT_ID = userdata.get('PROJECT_ID')

In [ ]:
# setting PROJECT ID
magics.context.project = PROJECT_ID

# Functions

In [ ]:
def evaluate(y_true, y_pred, y_prob=None, name="model"):
    acc = accuracy_score(y_true, y_pred)
    p, r, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="binary", zero_division=0)
    auc = roc_auc_score(y_true, y_prob) if (y_prob is not None and len(np.unique(y_true))>1) else np.nan
    print(f"{name:>14} | acc={acc:.3f}  prec={p:.3f}  rec={r:.3f}  f1={f1:.3f}  auc={auc:.3f}")
    return {"acc":acc, "prec":p, "rec":r, "f1":f1, "auc":auc}

In [ ]:
def best_threshold(y_true, y_prob):
    ths = np.linspace(0.05, 0.95, 19)
    scores = []
    for t in ths:
        pred = (y_prob >= t).astype(int)
        f1 = f1_score(y_true, pred, zero_division=0)
        scores.append((t, f1))
    t_star, f1_star = max(scores, key=lambda x: x[1])
    return t_star, f1_star

In [ ]:
def station_from_dummies(row):
    if not station_cols:
        return None
    ix = np.argmax(row[station_cols].values)
    return station_cols[ix].replace("station_", "")

# "Will it snow tomorrow?" - The time traveler asked
The following dataset contains climate information from over 9000 stations accross the world. The overall goal of these subtasks will be to predict whether it will snow tomorrow 20 years ago. So if today is 1 April 2025 then the weather we want to forecast is for the 2 April 2005. You are supposed to solve the tasks using Big Query, which can be used in the Jupyter Notebook like it is shown in the following cell. For further information and how to use BigQuery in Jupyter Notebook refer to the Google Docs.

The goal of this test is to test your coding knowledge in Python, BigQuery and Pandas as well as your understanding of Data Science. If you get stuck in the first part, you can use the replacement data provided in the second part.

In [ ]:
%%bigquery
SELECT
*,
FROM `bigquery-public-data.samples.gsod`
LIMIT 20

## Part 1

### 1. Task
Change the date format to 'YYYY-MM-DD' and select the data from 2000 till 2005 for station numbers including and between 725300 and 726300 , and save it as a pandas dataframe. Note the maximum year available is 2010.

In [ ]:
%%bigquery df
SELECT
  FORMAT_DATE('%Y-%m-%d', DATE(year, month, day)) AS date,
  *
FROM `bigquery-public-data.samples.gsod`
WHERE
  station_number BETWEEN 725300 AND 726300
  AND DATE(year, month, day) BETWEEN DATE('2000-01-01') AND DATE('2005-12-31')
ORDER BY station_number, date

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.date.min(), df.date.max()

In [ ]:
df.station_number.min(), df.station_number.max()

### 2. Task
From here you want to work with the data from all stations 725300 to 725330 that have information from 2000 till 2005.

In [ ]:
%%bigquery df
SELECT
  FORMAT_DATE('%Y-%m-%d', DATE(year, month, day)) AS date,
  *
FROM `bigquery-public-data.samples.gsod`
WHERE
  station_number BETWEEN 725300 AND 725330
  AND DATE(year, month, day) BETWEEN DATE('2000-01-01') AND DATE('2005-12-31')
ORDER BY station_number, date

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.date.min(), df.date.max()

In [ ]:
df.station_number.min(), df.station_number.max()

Start by checking which year received the most snowfall in our data.

In [ ]:
df.info()

In [ ]:
%%bigquery snow_by_year
SELECT
  EXTRACT(YEAR FROM DATE(year, month, day)) AS year,
  SUM(CAST(snow_depth AS INT64))            AS snow_depth
FROM `bigquery-public-data.samples.gsod`
WHERE
  station_number BETWEEN 725300 AND 725330
  AND DATE(year, month, day) BETWEEN DATE('2000-01-01') AND DATE('2005-12-31')
GROUP BY year
ORDER BY snow_depth DESC, year ASC

In [ ]:
snow_by_year

Add an additional field that indicates the daily change in snow depth measured at every station. And identify the station and day for which the snow depth increased the most.  

In the dataset, the `snow_depth` column contains many missing values. To retain as much temporal information as possible, I decided to replace missing values with `0`. This way, we avoid dropping large parts of the time series and can still work with a complete dataset for modeling.

A potential refinement would be to distinguish between *true zero snow depth* and *missing measurements*. For example, if the boolean feature `snow` indicates snowfall on a given day but `snow_depth` is missing, we could add an additional indicator column (`snow_depth_reported`) to flag whether the value was actually measured or imputed. This would preserve information about measurement quality and could improve model performance.


In [ ]:
%%bigquery df
WITH base AS (
  SELECT
    FORMAT_DATE('%Y-%m-%d', DATE(year, month, day)) AS date,
    *,
    IFNULL(snow_depth, 0) AS snow_depth_clean   -- missing values -> 0
  FROM `bigquery-public-data.samples.gsod`
  WHERE
    station_number BETWEEN 725300 AND 725330
    AND DATE(year, month, day) BETWEEN DATE('2000-01-01') AND DATE('2005-12-31')
),
diffs AS (
  SELECT
    *,
    snow_depth_clean - LAG(snow_depth_clean) OVER (PARTITION BY station_number ORDER BY date) AS daily_change_snow_depth
  FROM base
)
SELECT
  *
FROM diffs
ORDER BY daily_change_snow_depth DESC

In [ ]:
df.shape

In [ ]:
df.head(1)

The highest increase of snow depth was measured by station 725300 on Jan. 22nd  2005.

Do further checks on the remaining dataset, clean or drop data depending on how you see appropriate.

In [ ]:
# setting date as datetime
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df = df.sort_values(by=['station_number', 'date'])

In [ ]:
# target: snow tomorrow (0/1)
df['snow_tomorrow'] = (
    df.groupby('station_number')['snow']
      .shift(-1)                     # tomorrow
      .astype('Int64')               # int
)

In [ ]:
# Drop rows where target is NaN
df = df.dropna(subset=['snow_tomorrow'])

Since the dataset only contains 10 unique stations, I included station_number as a categorical feature via one-hot encoding. This allows the model to capture station-specific snowfall likelihoods. With a small number of categories this does not introduce a high-dimensional feature space.

In [ ]:
# checking if all stations have enough data
df.groupby("station_number")["date"].agg(["min","max","count"])

In [ ]:
# Station dummies
station_dummies = pd.get_dummies(df['station_number'], prefix='station')

# concat with df
df = pd.concat([df, station_dummies], axis=1)

# dropping original station_number
df = df.drop(columns=['station_number'])

In [ ]:
# Making a cyclical feature out of month (1–12)
df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
df['month_cos'] = np.cos(2 * np.pi * df['month']/12)

In [ ]:
# dropping unnecessary columns or columns with too many NaN values
df = df.drop(columns=[
    "wban_number",
    "year",
    "month",
    "day",
    "mean_station_pressure",
    "num_mean_station_pressure_samples",
    "min_temperature",
    "min_temperature_explicit",
    "snow_depth",
    "max_gust_wind_speed",
    "num_mean_sealevel_pressure_samples"
])

In [ ]:
# Simple imputation
for col in ['mean_dew_point', 'mean_wind_speed', 'max_temperature', 'total_precipitation', 'mean_sealevel_pressure', 'mean_visibility']:
    df[col] = df[col].fillna(df[col].median())

In [ ]:
df.info()

In [ ]:
# dropping the remained NaNs
df = df.dropna()

In [ ]:
df.shape

### 3. Task
Now it is time to split the data, into a training, evaluation and test set. As a reminder, the date we are trying to predict snow fall for should constitute your test set.

In [ ]:
str(dt.datetime.today()- dt.timedelta(days=20*365)).split(' ')[0]

The official challenge asks us to predict tomorrow’s snowfall 20 years ago. To reflect this, I defined the test set as all data from the cutoff date 20 years ago (≈2005). Training and validation use only earlier years (2000-2004), ensuring no data leakage.

In [ ]:
# Cutoff: today - 20 years
cutoff_date = (dt.datetime.today() - dt.timedelta(days=20*365)).date()
print("Target test date:", cutoff_date)

# Test: exact day
test_df = df[df["date"] == pd.to_datetime(cutoff_date)].copy()

# Train+Val: all data before
train_val_df = df[df["date"] < pd.to_datetime(cutoff_date)].copy()

In [ ]:
# train-val-split
train_df = train_val_df[train_val_df["date"] <= "2003-12-31"].copy()
val_df = train_val_df[(train_val_df["date"] > "2003-12-31") &
                      (train_val_df["date"] <= "2004-12-31")].copy()

print("Train:", train_df["date"].min(), "→", train_df["date"].max(), len(train_df))
print("Val:  ", val_df["date"].min(), "→", val_df["date"].max(), len(val_df))
print("Test:", test_df["date"].min(), "→", test_df["date"].max(), len(test_df))

## Part 2
If you made it up to here all by yourself, you can use your prepared dataset to train an algorithm of your choice to forecast whether it will snow on the following date for each station in this dataset:

In [ ]:
str(dt.datetime.today()- dt.timedelta(days=20*365)).split(' ')[0]

You are allowed to use any library you are comfortable with such as sklearn, tensorflow, keras etc.
If you did not manage to finish part one feel free to use the data provided in 'coding_challenge.csv' Note that this data does not represent a solution to Part 1.

In [ ]:
target = "snow_tomorrow"

In [ ]:
# choose feature columns (everything numeric except date & target)
drop_cols = ["date", target]
X_cols = [c for c in train_df.columns if c not in drop_cols]

In [ ]:
X_train, y_train = train_df[X_cols], train_df[target].astype(int)
X_val, y_val = val_df[X_cols], val_df[target].astype(int)
X_test = test_df[X_cols]
y_test = test_df[target].astype(int) if target in test_df.columns else None  # might be hidden for a real test

print("n_train:", len(X_train), "n_val:", len(X_val), "n_test:", len(X_test))
print("Positive rate (train/val):", y_train.mean().round(3), y_val.mean().round(3))

### Baseline

In [ ]:
# Always-0 baseline
evaluate(y_val, np.zeros_like(y_val), name="always_0")

The naive baseline of always predicting ‘no snow tomorrow’ achieves ~93% accuracy due to class imbalance, but completely fails to identify snow days (F1 = 0, Recall = 0). This underlines the need for more informative models and alternative metrics such as recall, precision and F1.

In [ ]:
# "snow today = snow tomorrow" baseline (uses today's snow as predictor)
evaluate(y_val, val_df["snow"].values.astype(int), name="snow_today")

The baseline of simply assuming that snowfall tomorrow equals snowfall today achieves ~88% accuracy. While worse than the trivial always-0 baseline in terms of accuracy, it at least captures ~19% of true snow days (recall). However, the precision and F1 remain low, highlighting that more sophisticated models are needed.

### Logistic Regression

In [ ]:
# scaling
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_val_s   = scaler.transform(X_val)
X_test_s  = scaler.transform(X_test)

In [ ]:
# Logistic Regression (with scaling, balanced classes)
logit = LogisticRegression()
logit.fit(X_train_s, y_train)

In [ ]:
val_prob_logit = logit.predict_proba(X_val_s)[:,1]

In [ ]:
t_logit, _ = best_threshold(y_val, val_prob_logit)

In [ ]:
# Validation performance
evaluate(y_val, (val_prob_logit>=t_logit).astype(int), val_prob_logit, name="logit")

In [ ]:
test_prob_logit = logit.predict_proba(X_test_s)[:,1]

In [ ]:
test_pred_logit = (test_prob_logit >= t_logit).astype(int)

In [ ]:
# recover station id from dummies (if original station_number was dropped)
station_cols = [c for c in X_test.columns if c.startswith("station_")]

In [ ]:
out = test_df.copy()
out["station"] = out.apply(station_from_dummies, axis=1)
out = out[["date", "station", "snow_tomorrow"]].copy()
out["p_logit"] = test_prob_logit
out["yhat_logit"] = test_pred_logit

In [ ]:
out = out.sort_values("station").reset_index(drop=True)
out

On the final test day (2005-08-29), snowfall occurred at only one station (725330). The logistic regression model correctly identified this snow event (recall = 100%), but at the cost of many false positives (6 out of 7 predicted snowfalls did not occur). This highlights the classic precision–recall tradeoff in imbalanced classification problems: the model is sensitive to snow but not specific.